In [1]:
import pandas as pd
from mftool import Mftool
from mf_performance_details import mf_performance
from tqdm.notebook import tqdm

In [2]:
mf_df=pd.read_pickle("mf_data_popular.pkl.bz2",compression="bz2")

In [3]:
# mf_df.to_pickle("mf_data_popular.pkl.bz2",compression="bz2")

In [4]:
mf_df.head()

,scheme_code,scheme_name,Risk,scheme_category,popularity,rank,mf_category,mf_sub_category,fund_name
0,100033,Aditya Birla Sun Life Equity Advantage Fund -...,Moderate,Equity Scheme - Large & Mid Cap Fund,27,6392.0,Equity Scheme,Large & Mid Cap,Aditya Birla Sun Life Equity Advantage Fund
1,100034,Aditya Birla Sun Life Equity Advantage Fund -...,Moderate,Equity Scheme - Large & Mid Cap Fund,22,11298.0,Equity Scheme,Large & Mid Cap,Aditya Birla Sun Life Equity Advantage Fund
2,100037,Aditya Birla Sun Life Income Fund - Regular -...,Low to Moderate,Debt Scheme - Medium to Long Duration Fund,28,5383.0,Debt Scheme,Medium to Long Duration,Aditya Birla Sun Life Income Fund
3,100038,Aditya Birla Sun Life Income Fund - Growth - ...,Moderately High,Debt Scheme - Medium to Long Duration Fund,25,8528.0,Debt Scheme,Medium to Long Duration,Aditya Birla Sun Life Income Fund
4,100041,Aditya Birla Sun Life Liquid Fund -Institutio...,Moderately High,Debt Scheme - Liquid Fund,26,7459.0,Debt Scheme,Liquid,Aditya Birla Sun Life Liquid Fund


In [5]:
# dict_fund={"fund_name":[],"risk":[]}

In [6]:
# mf_risk_data=list(set(mf_df["fund_name"]))
# for mf in mf_risk_data:
#     risk=mf_df[mf_df["fund_name"]==mf]["Risk"].values[0]
#     dict_fund["fund_name"].append(mf)
#     dict_fund["risk"].append(risk)

In [7]:
pd.read_pickle("fund_risk_score.pkl.bz2")

,fund_name,risk
0,BOI AXA Bluechip Fund Direct Plan Growth,Very High
1,JM Dynamic Debt Fund (Regular),Moderately High
2,PGIM India Flexi Cap Fund,Low to Moderate
3,Navi Large Cap Equity Fund,Low to Moderate
4,NIPPON INDIA OVERNIGHT FUND,Low to Moderate
...,...,...
2524,Franklin India Bluechip Fund,Low to Moderate
2525,UTI SENSEX Exchange Traded Fund,Moderate
2526,Tata Fixed Maturity Plan Series 55 Scheme I,Moderate
2527,IDBI FOCUSED 30 EQUITY FUND,Moderately High


In [8]:
# pd.DataFrame(dict_fund).to_pickle("fund_risk_score.pkl.bz2",compression="bz2")

In [9]:
def get_response(df):
    try:
        json = df.to_json(orient='records')
        return Response(json, mimetype='application/json')
    except Exception as ex:
        print(ex,json)
        return get_exception_response(ex)

In [10]:
def get_exception_response(ex):
    return Response(ex,500)

In [11]:
# def performance_based_recommendation(mf_category=None,mf_sub_category=None,risk="Moderate",top_n=5,load_cache=True):
#     try:
        

#         top_funds["risk"]= top_funds["scheme_name"].apply(lambda x: get_mf_risk(x))
#         return get_response(top_funds[top_funds["risk"]==risk].head(top_n))
#     except Exception as ex:
#         return get_exception_response("Error occured, please try again.")

In [12]:
def get_popularity_based_funds(mf_category=None,mf_sub_category=None,risk=None,top_n=5,load_cache=True,user_mf_json=None):
    FILE_NAME="mf_data_popular.pkl.bz2"
    global mf_df
    if load_cache==True:
        if mf_df is None:
            mf_df=pd.read_pickle(FILE_NAME,compression="bz2")
    else:
        mf_df=calculate_popularity(FILE_NAME,user_mf_json)
    filter_df= mf_df.copy(deep=True)
    if mf_category is not None:
        filter_df=filter_df[filter_df["mf_category"]==mf_category]
    if mf_sub_category is not None:
        filter_df=filter_df[filter_df["mf_sub_category"]==mf_sub_category]    
    
    mf_perf=mf_performance()
    suggested_mf=list(set(filter_df["fund_name"]))
    if risk is not None:        
        filter_df["risk_score"]= filter_df["scheme_name"].apply(lambda x: mf_perf.get_mf_risk(x))
        print(filter_df["risk_score"])
        suggested_mf=list(set(filter_df[filter_df["risk_score"]==risk]["fund_name"]))[:5] 
    
    result=get_performance_data(suggested_mf,mf_category,mf_sub_category,risk,load_cache)
    result["risk"]= result["scheme_name"].apply(lambda x: mf_perf.get_mf_risk(x))
        
    return result.head(top_n)

In [13]:
def calculate_popularity(mf_data_file,user_mf_json):
    global mf_df
    if mf_df is None:
            mf_df=pd.read_pickle(mf_data_file,compression="bz2")   
    user_df=pd.read_json(user_mf_json)
    for index,mf in mf_df["scheme_code"].items():
        mf_df["popularity"].iloc[index]=len(user_df[user_df["mf_id"]==mf])
    mf_df['rank'] = mf_df['popularity'].rank(ascending = 0, method = 'first')
    return mf_df

In [14]:
def get_mf_details(scheme_name,mf_category=None,mf_sub_category=None,load_cache=True):
        selected_mf=None
        selected_ratio=-1
        result=pd.DataFrame(columns=['scheme_name', 'benchmark', 'latest NAV- Regular', 'latest NAV- Direct',\
       '1-Year Return(%)- Regular', '1-Year Return(%)- Direct',\
       '3-Year Return(%)- Regular', '3-Year Return(%)- Direct',\
       '5-Year Return(%)- Regular', '5-Year Return(%)- Direct'])

        for mf in mf_data["scheme_name"]:
            if scheme_name.lower() == mf.lower():
                ratio=100
            else:
                ratio=SequenceMatcher(a=scheme_name.lower(),b=mf.lower()).ratio()
            if ratio >selected_ratio:
                selected_ratio=ratio
                selected_mf=mf 
                if selected_ratio==100:
                    break
        if selected_ratio>-1:
            result=mf_data[mf_data["scheme_name"]==selected_mf].drop_duplicates()
        return result

In [15]:
def get_performance_data(fund_names,mf_category=None,mf_sub_category=None,risk=None,load_cache=True):
    mf_perf=mf_performance()
    df_list=[]
    
    for fund in tqdm(fund_names):
        df_perf=mf_perf.get_mf_details(fund,mf_category,mf_sub_category,risk,load_cache)
        df_list.append(df_perf)
    final_df=pd.concat(df_list)
    final_df=pd.concat(df_list).reset_index(drop=True)
    
    return final_df

In [16]:
df_data=get_popularity_based_funds(mf_category="Debt Scheme",risk="Moderate")

KeyboardInterrupt: 

In [ ]:
df_data.head()

In [ ]:
df_data=get_popularity_based_funds(mf_category="Equity Scheme")

In [ ]:
df_data

In [ ]:
def get_mf_risk(scheme_name,mf_df,default="Moderate"):
    selected_mf=None
    selected_ratio=-1    
    mf_risk_score= mf_df[mf_df["fund_name"].str.lower()==str(scheme_name).lower()]["Risk"].to_list()
    if len(mf_risk_score)>=1:
        return str(mf_risk_score[0])
    elif len(mf_risk_score)==0:
        ratio=SequenceMatcher(a=scheme_name.lower(),b=mf.lower()).ratio()
        for mf in mf_df["fund_name"].str.lower():
            ratio=SequenceMatcher(a=scheme_name.lower(),b=mf.lower()).ratio()
            if ratio >selected_ratio:
                selected_ratio=ratio
                selected_mf=mf 
        mf_risk_score= mf_df[mf_df["fund_name"].str.lower()==str(selected_mf).lower()]["Risk"].to_list()
        return mf_risk_score[0]
    else:
        return str(mf_risk_score)

In [ ]:
def get_mf_details(self,scheme_name,mf_category=None,mf_sub_category=None,load_cache=True):
        selected_mf=None
        selected_ratio=-1
        result=pd.DataFrame(columns=['scheme_name', 'benchmark', 'latest NAV- Regular', 'latest NAV- Direct',\
       '1-Year Return(%)- Regular', '1-Year Return(%)- Direct',\
       '3-Year Return(%)- Regular', '3-Year Return(%)- Direct',\
       '5-Year Return(%)- Regular', '5-Year Return(%)- Direct'])
        if self.mf_perform_data is None:
            self.mf_perform_data=self.mutual_fund_performance(mf_category,mf_sub_category,load_cache)
        for mf in self.mf_perform_data["scheme_name"]:
            if scheme_name.lower() == mf.lower():
                ratio=100
            else:
                ratio=SequenceMatcher(a=scheme_name.lower(),b=mf.lower()).ratio()
            if ratio >selected_ratio:
                selected_ratio=ratio
                selected_mf=mf 
                if selected_ratio==100:
                    break
        if selected_ratio>-1:
            result=self.mf_perform_data[self.mf_perform_data["scheme_name"]==selected_mf].drop_duplicates()
        return result

In [ ]:
load_cache=True
mf_category="Debt Scheme"
risk="Moderate"
mf_sub_category=None
top_n=5

In [ ]:
FILE_NAME="mf_data_popular.pkl.bz2"
global mf_df
if load_cache==True:
    if mf_df is None:
        mf_df=pd.read_pickle(FILE_NAME,compression="bz2")
else:
    mf_df=calculate_popularity(FILE_NAME,user_mf_json)
filter_df= mf_df.copy(deep=True)
if mf_category is not None:
    filter_df=filter_df[filter_df["mf_category"]==mf_category]
if mf_sub_category is not None:
    filter_df=filter_df[filter_df["mf_sub_category"]==mf_sub_category]
if risk is not None:
    filter_df=filter_df[filter_df["Risk"]==risk]
filter_df=filter_df.sort_values(by="rank",ascending=True).head(top_n)
result=get_performance_data(filter_df["fund_name"].to_list(),mf_category="Debt Scheme",risk="Moderate")

In [ ]:
result

In [ ]:
result[result["risk"]==risk].head(top_n)